In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, datasets, models
from dotenv import load_dotenv
import os
import pathlib
from tqdm.auto import tqdm
import time

load_dotenv('../.env')

FLASH_DIR = pathlib.Path(os.getenv("FLASH_DIR"))
NON_FLASH_DIR = pathlib.Path(os.getenv("NON_FLASH_DIR"))
IMAGES_DIR = pathlib.Path(os.getenv("IMAGES_DIR"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [35]:
training_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.CenterCrop(256),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])  

dataset = datasets.ImageFolder(root=IMAGES_DIR, transform=training_transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size  

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

test_dataset.dataset.transform = test_transform

train_labels = [dataset.targets[i] for i in train_dataset.indices]

class_counts = np.bincount(train_labels)
class_weights = [1.0 / count for count in class_counts]
print(f"Class weights: {class_weights}")

sample_weights = [class_weights[label] for label in train_labels]
sample_weights = torch.DoubleTensor(sample_weights)

sampler = torch.utils.data.WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, sampler=sampler)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)



Class weights: [0.001652892561983471, 8.309098462816784e-05]


In [38]:
class FlashbangModel(nn.Module):
    def __init__(self):
        super(FlashbangModel, self).__init__()
        
        self.model = models.resnet18(pretrained=True)
        
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, 2)
        
    def forward(self, x):
        return self.model(x)

In [39]:
model = FlashbangModel().to(device)

class_weights = torch.tensor([1.0, 20.0]).to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

c:\Users\baig_\.pyenv\pyenv-win\versions\3.12.8\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\baig_\.pyenv\pyenv-win\versions\3.12.8\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\baig_/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 71.8MB/s]


In [ ]:
def train_model(model, train_loader, criterion, optimizer, device, epochs=10):
    model.train()
    
    start_time = time.time()   
    
    for epoch in tqdm(range(epochs)):
        loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
        total_loss = loss / len(train_loader)
        accuracy = correct / total * 100
    
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {total_loss:.4f} - Accuracy: {accuracy:.2f}")
        
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training took {elapsed_time:.2f} seconds")

In [42]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    
    loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
        accuracy = correct / total * 100
    
        print(f"Test Accuracy: {accuracy:.2f}")

In [47]:
train_model(model, train_loader, loss_fn, optimizer, device, epochs=10)
evaluate_model(model, test_loader, loss_fn, device)

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [48]:
!nvidia-smi

Sat Feb 22 21:23:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.16                 Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti   WDDM  |   00000000:08:00.0  On |                  N/A |
| 32%   55C    P5             51W /  350W |    5887MiB /  12288MiB |     39%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [49]:
device

device(type='cuda')